<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_07_2_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 7: LangChain: Agents**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 7 Material

* Part 7.1: Introduction to LangChain Agents [[Video]](https://www.youtube.com/watch?v=J5Vr___lSSs) [[Notebook]](t81_559_class_07_1_agents.ipynb)
* **Part 7.2: Understanding LangChain Agent Tools** [[Video]](https://www.youtube.com/watch?v=qMquBmteYw4) [[Notebook]](t81_559_class_07_2_tools.ipynb)
* Part 7.3: LangChain Retrival and Search Tools [[Video]](https://www.youtube.com/watch?v=NB5qGPLoBBE) [[Notebook]](t81_559_class_07_3_search_tools.ipynb)
* Part 7.4: Constructing LangChain Agents [[Video]](https://www.youtube.com/watch?v=OJe5oHvrdHk) [[Notebook]](t81_559_class_07_4_more_agent.ipynb)
* Part 7.5: Custom Agents [[Video]](https://www.youtube.com/watch?v=IsJemVYSEdc) [[Notebook]](t81_559_class_07_5_custom_agent.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai langchain_experimental duckduckgo-search langchainhub

Note: not using Google CoLab


# 7.2: LangChain Agent Tools


LangChain agents are versatile entities designed to perform specific tasks autonomously. Central to their functionality are [tools](https://python.langchain.com/v0.1/docs/modules/tools/), which are specialized components that agents can utilize to accomplish their objectives. These tools can range from data retrieval and processing utilities to interactive interfaces for user engagement. By leveraging these tools, LangChain agents can efficiently execute complex workflows, automate routine tasks, and provide intelligent solutions tailored to user needs. Whether it's querying databases, parsing documents, or interacting with APIs, the strategic use of tools enables LangChain agents to enhance productivity and deliver precise outcomes.


Large Language Models (LLMs) inherently lack access to real-time information such as the current date and time, stock market data, and breaking news. This limitation stems from their design, which relies on pre-existing datasets that do not include ongoing updates. Additionally, LLMs are not equipped to perform mathematical calculations directly, which can restrict their utility in scenarios requiring precise numerical operations. To overcome these constraints, LLMs can employ specialized tools. For instance, search engine tools enable LLMs to retrieve the latest information from the web, ensuring up-to-date responses. Similarly, tools designed for mathematical computation can assist LLMs in accurately processing and solving mathematical problems, thereby enhancing their overall capability and accuracy.

* [Available Langchain Tools](https://python.langchain.com/v0.2/docs/integrations/tools/)
* [Available Langchain Toolkits](https://python.langchain.com/v0.2/docs/integrations/toolkits/)

## Tools for Math


We will begin our exploration of LangChain tools by using a tool specifically designed for math. Large Language Models (LLMs), like ChatGPT, are generally poor at performing mathematical calculations without the assistance of specialized tools. This is because LLMs are trained primarily on textual data and lack the precision required for accurate arithmetic operations. Consequently, they often produce inaccurate results when asked to perform math independently. To see how a LLM actually performs mathematics, I asked ChatGPT and it gave a decent high-level summary.

> Large Language Models (LLMs), like me, do not inherently perform arithmetic calculations the same way a calculator or dedicated algorithm would. Instead, we generate responses based on patterns in the data we've been trained on. Here's a simplified explanation of how we handle such tasks:
>
> Pattern Recognition: During training, LLMs are exposed to vast amounts of text data, which includes examples of arithmetic and mathematical reasoning. We learn patterns and structures in these examples, enabling us to approximate calculations.
>
> Token Prediction: When asked to perform a calculation, an LLM doesn't actually "calculate" in the traditional sense. Instead, it predicts the most likely sequence of tokens (numbers, in this case) that should follow based on the input. This prediction is influenced by the training data but does not involve real arithmetic operations.
>
> Approximation and Heuristics: For smaller or simpler calculations, the model might generate the correct answer because it has seen enough examples during training. For larger or more complex calculations, the model might generate an approximate answer or even make a guess based on learned patterns.
>
> For example, if you ask an LLM to multiply 872947493 by 7492374932, it will try to generate a plausible sequence of digits based on what it has seen in the training data, but this sequence is unlikely to be correct without an actual computational algorithm.
>
>Here’s a brief comparison of how a traditional method (e.g., a calculator or algorithm) and an LLM approach such a problem:
>
> * Traditional Method: Uses precise algorithms to perform each step of the multiplication (e.g., long multiplication or fast algorithms like the Karatsuba algorithm).
> * LLM Method: Predicts the next sequence of digits based on patterns and probabilities from the training data.
So, while an LLM might "attempt" to give an answer, it lacks the precision and algorithmic foundation to guarantee accuracy for complex arithmetic without dedicated computational tools.

To see this in action, let's ask an LLM to perform a mathematical operation. We will choose numbers that were unlikely in the LLM's training data.



In [2]:
from langchain_openai import ChatOpenAI

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )

print(llm.invoke("What is 8273 times 1821?"))

content='8273 times 1821 equals 15,086,213.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 17, 'total_tokens': 31}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None} id='run-3a675778-1103-4384-8350-8082c63cca8e-0' usage_metadata={'input_tokens': 17, 'output_tokens': 14, 'total_tokens': 31}


The resulting number appears reasonable, but that's the point. LLMs are trained to produce believable results, not necessarily correct ones. To verify the LLM, we'll have Python perform this calculation.

In [3]:
print(8273 * 1821)

15065133


In [4]:
print( abs(15065133 - 15055433 ) )


9700


We can see that the LLM was several thousand off. Unfortunately, LangChain does not include a calculator tool, at least as of summer 2024. We will look at two approaches. First, we will use the LangChain-provided PythonREPL. The following code shows how to use PythonREPL.

In [5]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()

python_repl.run("print(1+1)")

'2\n'

It begins by importing necessary modules and classes from the LangChain, LangChain OpenAI, and LangChain Community libraries.
A tool for executing Python commands (repl_tool) is created using the Tool class, with a description indicating its purpose as a Python shell that requires valid Python commands. The function python_repl.run is assigned to execute the commands.

The prompt for the agent is pulled from a hub, specified by the identifier "hwchase17/openai-functions-agent".

An agent is then created using the create_tool_calling_agent function, which takes the language model (llm), the tools (here, just repl_tool), and the prompt. This agent is wrapped in an AgentExecutor with the verbose mode enabled to provide detailed logs of the execution process.
Finally, the agent is invoked with an input command to calculate the product of 8273 and 1821.

In [6]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain.tools import StructuredTool
from pydantic import BaseModel

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
    model=MODEL,
    temperature=0.2,
    n=1
)

# Define a Pydantic schema for the input arguments
class PythonReplInput(BaseModel):
    input: str

# Define the tool using StructuredTool with an args_schema
def run_python_code(input: str, **kwargs):
    try:
        # Safely evaluate the input string as a Python expression
        result = eval(input)
        return str(result)
    except Exception as e:
        return str(e)

repl_tool = StructuredTool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=run_python_code,  # Use the custom Python REPL function
    args_schema=PythonReplInput  # Define the expected input schema
)

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [repl_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Pass a string Python command as input
result = agent_executor.invoke({"input": "What is 8273 * 1821?"})

print(result)


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langsmith/client.py:5515: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))




> Entering new AgentExecutor chain...

Invoking: `python_repl` with `{'input': '8273 * 1821'}`


15065133The result of \( 8273 \times 1821 \) is \( 15,065,133 \).

> Finished chain.
{'input': 'What is 8273 * 1821?', 'output': 'The result of \\( 8273 \\times 1821 \\) is \\( 15,065,133 \\).'}


## Create a Cusom Math Tool

The Python REPL tool we just used can execute any Python command. Therefore, it can be a security concern if we only wish to perform math calculations. In this section, we will see how to create a custom tool that can only perform basic math calculations.

In [7]:
from langchain_openai import OpenAI
import numpy as np
from langchain.chains.base import Chain

class SafeCalculator:
    def calculate(self, expression):
        try:
            # Evaluate the mathematical expression using NumPy
            result = eval(expression, {"__builtins__": None}, {"np": np})
            return result
        except Exception as e:
            return str(e)

class CalculatorChain(Chain):
    calculator: SafeCalculator

    def _call(self, inputs):
        expression = inputs["expression"]
        result = self.calculator.calculate(expression)
        return {"result": result}

    @property
    def input_keys(self):
        return ["expression"]

    @property
    def output_keys(self):
        return ["result"]

# Initialize the safe calculator tool
safe_calculator = SafeCalculator()

# Example usage
chain = CalculatorChain(calculator=safe_calculator)
expression = "3 * (2 + 5) / 7"
inputs = {"expression": expression}
result = chain(inputs)
print(f"Result: {result['result']}")


Result: 3.0


<ipython-input-7-34b3ab86af62>:38: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = chain(inputs)


In [10]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain.tools import StructuredTool
from pydantic import BaseModel

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
    model=MODEL,
    temperature=0.2,
    n=1
)

# Define a Pydantic schema for the input arguments
class MathExpressionInput(BaseModel):
    input: str

# Define a safe calculator function
def safe_calculator(input: str, **kwargs):
    try:
        # Safely evaluate the input math expression
        result = eval(input)
        return str(result)
    except Exception as e:
        return str(e)

# Define the tool using StructuredTool with args_schema
safe_math_tool = StructuredTool(
    name="safe_calc",
    description="A math calculator used to evaluate mathematical expressions. Input should be a valid math expression, similar to Python.",
    func=safe_calculator,  # Use the safe calculator function
    args_schema=MathExpressionInput  # Define the expected input schema
)

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [safe_math_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Pass a mathematical expression as input
result = agent_executor.invoke({"input": "8273 * 1821"})

print(result)


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `safe_calc` with `{'input': '8273 * 1821'}`


15065133The result of \( 8273 \times 1821 \) is \( 15,065,133 \).

> Finished chain.
{'input': '8273 * 1821', 'output': 'The result of \\( 8273 \\times 1821 \\) is \\( 15,065,133 \\).'}
